In [1]:
# !apt-get update
# !apt-get install -y libsm6 libxext6 libxrender-dev
# !pip install opencv-python

In [2]:
#!pip install openmim
#!mim install mmdet

In [3]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [4]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/_base_/my_seg/swin_base.py')

cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 4

cfg.seed = 2021
cfg.gpu_ids = [0]

cfg.work_dir = './work_dirs/real_swin_base'

cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
cfg.model.roi_head.bbox_head[2].num_classes = 10
cfg.model.roi_head.mask_head.num_classes = 10

# image_size 변환 - default : (1024, 1024), 적용시 (2048, 2048)
# cfg.train_pipeline[2].img_scale=(2048,2048)
# cfg.test_pipeline[1].img_scale=(2048,2048)
# cfg.val_pipeline[2].img_scale=(2048,2048)

# neck 변환 - default : FPN, 적용시 PAFPN
# cfg.model.neck=dict(type = 'PAFPN',
#                     in_channels=[96, 192, 384, 768],
#                     out_channels=256,
#                     num_outs=6)

# anchor size, 비율 변환 - default : scales = [8], ratios=[0.5, 1.0, 2.0], strides=[2, 4, 8, 16, 32, 64])
# cfg.model.rpn_head.anchor_generator=dict(
#             type='AnchorGenerator',
#             scales=[8],
#             ratios=[0.33, 0.5, 1.0, 2.0, 3.0],
#             strides=[2, 4, 8, 16, 32, 64])

# rpn_proposal 변환 - default : 2000 적용시 : 1000
# cfg.model.train_cfg.rpn.rpn_proposal=dict(
#             nms_pre=1000,
#             max_per_img=1000,
#             nms=dict(type='nms', iou_threshold=0.7),
#             min_bbox_size=0)

# loss변환 - default : cross entropy 적용시 focal loss
# cfg.model.rpn_head.loss_cls = dict(type='FocalLoss', loss_weight=1.0)
# cfg.model.roi_head.bbox_head[0].loss_cls = dict(
#     type='FocalLoss', loss_weight=1.0)
# cfg.model.roi_head.bbox_head[1].loss_cls = dict(
#     type='FocalLoss', loss_weight=1.0)
# cfg.model.roi_head.bbox_head[2].loss_cls = dict(
#     type='FocalLoss', loss_weight=1.0)

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=-1, interval=1)
cfg.runner = dict(type='EpochBasedRunner', max_epochs=30)
cfg.lr_config =  dict(
    policy='CosineAnnealing',
    warmup='linear',
    warmup_iters=300,
    warmup_ratio=0.001,
    min_lr_ratio=1e-4,
)

# wandb log설정
cfg.log_config = dict(
    interval=100,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='EpochWandbLogger',
        init_kwargs=dict(
            project='Swin-seg',
            name='swin_base'))
])
cfg.workflow = workflow = [('train', 1),('val',1)]

In [5]:
# build_dataset
datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val_loss)] # 

loading annotations into memory...
Done (t=3.72s)
creating index...
index created!
loading annotations into memory...
Done (t=1.28s)
creating index...
index created!


In [6]:
datasets[0]


CocoDataset Train dataset with number of images 2616, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 2225  | 1 [Paper]     | 7448  | 2 [Paper pack]  | 527   | 3 [Metal]   | 449   | 4 [Glass]    | 488   |
| 5 [Plastic]       | 2472  | 6 [Styrofoam] | 1074  | 7 [Plastic bag] | 6114  | 8 [Battery] | 50    | 9 [Clothing] | 141   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [7]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

/opt/ml/segmentation/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_base_patch4_window7_224.pth" to /opt/ml/.cache/torch/hub/checkpoints/swin_base_patch4_window7_224.pth


2021-10-20 06:32:47,331 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias

2021-10-20 06:32:47,447 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2021-10-20 06:32:47,507 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2021-10-20 06:32:47,515 - mmdet - INFO - initialize ConvFCBBoxHead with init_cfg [{'type': 'Nor

In [8]:
# 모델 학습
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...


2021-10-20 06:32:50,425 - mmdet - INFO - Start running, host: root@65f88b0d31f6, work_dir: /opt/ml/segmentation/mmdetection/work_dirs/real_swin_base
2021-10-20 06:32:50,427 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) EpochWandbLogger                   
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) EvalHook                           
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) EpochWandbLogger                   
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) EvalHook                           
(LO

Done (t=0.90s)
creating index...
index created!


wandb: Currently logged in as: ksm0517 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


RuntimeError: CUDA out of memory. Tried to allocate 58.00 MiB (GPU 0; 31.75 GiB total capacity; 6.17 GiB already allocated; 57.75 MiB free; 6.37 GiB reserved in total by PyTorch)